# TP3 - Views, Updates and Design

The objectives for this TP are:

1. Create and use SQL Views
2. Update database content
3. Design the database schema for a Social Network

___

For the first 2 parts we will use the **`wine.db`** database and the Tables created in TP2.

A reminder of the wine database schema:

<center>**Master1**</center>

|*Attribute*|         *Description*          |
| -------   |--------------------------------|
| NV        | Wine number                    |
| CRU       | Vineyard or group of vineyards |
| DEGRE     | Alcohol content                |
| MILL      | Vintage year                   |
| QTE       | Number of bottles harvested    |
| NP        | Producer number                |
| NOM       | Producer's last name           |
| PRENOM    | Producer's first name          |
| REGION    | Production region              |

<center>**Master2**</center>

|*Attribute*|                         *Description*                  |
| -------   |--------------------------------------------------------|
| NV        | Wine number                                            |
| CRU       | Vineyard or group of vineyards                         |
| DEGRE     | Alcohol content                                        |
| MILL      | Vintage year                                           |
| DATES     | Buying date                                            |
| LIEU      | Place where the wine was sold                          |
| QTE       | Number of bottles bought                               |
| NB        | Client (buveur) number                                 |
| NOM       | Client's last name                                     |
| PRENOM    | Client's first name                                    |
| TYPE      | Type of client by volume of purchases                  |
| REGION    | Administrative Region (different to production region) |


In [151]:
import sqlite3
from IPython.utils.traitlets import Bool, Int, Unicode

In [152]:
def printSchema(connection):
    ### Source: http://stackoverflow.com/a/35092773/4765776
    for (tableName,) in connection.execute(
        """
        select NAME from SQLITE_MASTER where TYPE='table' order by NAME;
        """
    ):
        print("{}:".format(tableName))
        for (
            columnID, columnName, columnType,
            columnNotNull, columnDefault, columnPK,
        ) in connection.execute("pragma table_info('{}');".format(tableName)):
            print("  {id}: {name}({type}){null}{default}{pk}".format(
                id=columnID,
                name=columnName,
                type=columnType,
                null=" not null" if columnNotNull else "",
                default=" [{}]".format(columnDefault) if columnDefault else "",
                pk=" *{}".format(columnPK) if columnPK else "",
            ))

In [153]:
conn = sqlite3.connect('wine.db')
c = conn.cursor()
print("Database schema:")
printSchema(conn)

Database schema:
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(DATE)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
NB_MASTER2:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
NP_MASTER1:
  0: NP(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: REGION(TEXT)
NV_MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
NV_MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
NV_NB_DATES_MASTER2:
  0: NV(NUM)
  1: NB(NUM)
  2: DATES(NUM)
  3: QTE(NUM)
  4: LIEU(TEXT)
  5: REGION(TEXT)
NV_NP_MASTER1:
  0: NV(NUM)
  1: NP(NUM)
  2: QTE(NUM)
RBA:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
RBB:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
sqlite_sequence:
  0: name()
  1: seq()


Again, we will use **`%%sql`** magic for our queries

In [154]:
%load_ext sql
%sql sqlite:///wine.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


'Connected: None@wine.db'

Recreate the Normalized Tables from **Master1** and **Master2** as you did in the TP2

In [155]:
%%sql
DROP TABLE IF EXISTS NV_NP_MASTER1;
--- Create NV_NP_MASTER1 table
CREATE TABLE NV_NP_MASTER1 AS
SELECT NV, NP, QTE
FROM MASTER1
WHERE NV IS NOT NULL AND NP IS NOT NULL;

DROP TABLE IF EXISTS NV_MASTER1;
--- Create NV_MASTER1 table
CREATE TABLE NV_MASTER1 AS
SELECT DISTINCT NV, CRU, DEGRE, MILL
FROM MASTER1
WHERE NV IS NOT NULL;

DROP TABLE IF EXISTS NP_MASTER1;
--- Create NP_MASTER1 table
CREATE TABLE NP_MASTER1 AS
SELECT DISTINCT NP, NOM, PRENOM, REGION
FROM MASTER1
WHERE NP IS NOT NULL;

DROP TABLE IF EXISTS NV_NB_DATES_MASTER2;
--- Create NV_NB_DATES_MASTER2 table
CREATE TABLE NV_NB_DATES_MASTER2 AS
SELECT NV, NB, DATES, QTE, LIEU, REGION
FROM MASTER2
WHERE NV IS NOT NULL AND NB IS NOT NULL AND DATES IS NOT NULL;

DROP TABLE IF EXISTS NV_MASTER2;
--- Create NV_MASTER2 table
CREATE TABLE NV_MASTER2 AS
SELECT DISTINCT NV, CRU, DEGRE, MILL
FROM MASTER2
WHERE NV IS NOT NULL;

DROP TABLE IF EXISTS NB_MASTER2;
--- Create NB_MASTER2 table
CREATE TABLE NB_MASTER2 AS
SELECT DISTINCT NB, NOM, PRENOM, TYPE
FROM MASTER2
WHERE NB IS NOT NULL;

Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

___
# PART I: CREATE AND USE VIEWS

A view is a virtual table based on the result-set of an SQL statement. Views are stored in the database with an associated name.

Views are created following the syntax:

```mysql
CREATE VIEW view_name AS
SELECT column1, column2.....
FROM table_name
WHERE [condition];
```

An useful command:

```mysql
DROP VIEW IF EXISTS view_name;
```


__Warning:__ Use `DROP` with caution (only drop something if you are sure)

__Note:__ You will find some cells marked as "Test" that will help you check your work. Do NOT modify them. 

#### Exercise 1.1

Create a view "**bons_buveurs**" with the clients (buveurs) of type 'gros' or 'moyen'.

In [156]:
%%sql
DROP VIEW IF EXISTS bons_buveurs;
CREATE VIEW bons_buveurs AS
SELECT NB, NOM, PRENOM, TYPE
FROM NB_MASTER2
WHERE TYPE IN ('gros', 'moyen');

Done.
Done.


[]

In [157]:
# Test
%sql SELECT * FROM bons_buveurs ORDER BY nb LIMIT 5;

Done.


NB,NOM,PRENOM,TYPE
2,Artaud,Antonin,moyen
3,Aron,Raymond,gros
4,Apollinaire,Guillaume,moyen
6,Arrabal,Fernando,gros
7,Anouilh,Jean,moyen


#### Exercise 1.2

Create the view "**buveurs_asec**" with clients (buveurs) who have not bought any wine.

In [158]:
%%sql
DROP VIEW IF EXISTS buveurs_asec;
CREATE VIEW buveurs_asec AS
SELECT NB_MASTER2.NB, NOM, PRENOM, TYPE
FROM NB_MASTER2
LEFT JOIN NV_NB_DATES_MASTER2 ON NV_NB_DATES_MASTER2.NB=NB_MASTER2.NB
WHERE QTE IS NULL

Done.
Done.


[]

In [159]:
# Test
%sql SELECT * FROM buveurs_asec ORDER BY nb LIMIT 5;

Done.


NB,NOM,PRENOM,TYPE
11,Breton,Andre,petit
13,Barthes,Roland,moyen
16,Balzac,Honore de,moyen
18,Celine,Louis Ferdinand,gros
20,Chateaubriand,Francois-Rene de,moyen


#### Exercise 1.3

Create the view "**buveurs_achats**" complementary to the previous one.

In [160]:
%%sql
DROP VIEW IF EXISTS buveurs_achats;
CREATE VIEW buveurs_achats AS
SELECT NB, NOM, PRENOM, TYPE
FROM NB_MASTER2
EXCEPT
SELECT NB_MASTER2.NB, NOM, PRENOM, TYPE
FROM NB_MASTER2
LEFT JOIN NV_NB_DATES_MASTER2 ON NV_NB_DATES_MASTER2.NB=NB_MASTER2.NB
WHERE QTE IS NULL

Done.
Done.


[]

In [161]:
# Test
%sql SELECT * FROM buveurs_achats ORDER BY nb LIMIT 5;

Done.


NB,NOM,PRENOM,TYPE
1,Aristote,None,petit
2,Artaud,Antonin,moyen
3,Aron,Raymond,gros
4,Apollinaire,Guillaume,moyen
5,Audiberti,Jacques,petit


#### Exercise 1.4

Create the view "**q83pl**" (LIEU, CRU, QTE_BUE) that provides by LIEU and CRU the total quantities bought in 1983 by all the clients (buveurs).

In [162]:
%%sql
DROP VIEW IF EXISTS q83pl;
CREATE VIEW q83pl AS
SELECT LIEU, CRU, SUM(QTE) AS QTE_BUE, DATES
FROM NV_NB_DATES_MASTER2
INNER JOIN NV_MASTER2 ON NV_MASTER2.NV=NV_NB_DATES_MASTER2.NV
WHERE DATES LIKE '1983%'
GROUP BY LIEU, CRU;

Done.
Done.


[]

In [163]:
# Test
%sql SELECT * FROM q83pl;

Done.


LIEU,CRU,QTE_BUE,DATES
CAEN,Seyssel,3,1983-02-21
LILLE,Pommard,5,1983-06-29
LYON,Beaujolais Villages,10,1983-06-06
LYON,Julienas,2,1983-12-25
PARIS,Beaujolais Primeur,4,1983-03-10
PARIS,Coteaux du Tricastin,1,1983-12-31
PARIS,Pouilly Vinzelles,3,1983-02-14
RENNES,Mercurey,1,1983-12-31
ROCQUENCOURT,Beaujolais Villages,260,1983-05-05
ROCQUENCOURT,Saint Amour,80,1983-05-05


#### Exercise 1.5

Can we define the same view with ascending order over the attribute "QTE"? Provide an explanation for your answer.

___
# PART II: UPDATE DATABASE CONTENT

The syntax for the `UPDATE` operation is:

```sql
UPDATE table_name
SET column1 = value1, column2 = value2...., columnN = valueN
WHERE [condition];
```

The syntax for the `INSERT` operation is:

```sql
INSERT INTO table_name [(column1, column2, column3,...columnN)]  
VALUES (value1, value2, value3,...valueN);
```

Database updates are commited automatically in Jupyter/Python. _Transactions_ are an option to control and reverse changes. Additionally we can just reload a backup of the database (NOT an option in deployed systems)

__Note:__ Different to other Database Management Systems, SQLite views are read-only and so you can not execute a `DELETE`, `INSERT` or `UPDATE` statement on a view.

#### Exercise 2.1

Create a table "**RBB**" with the same schema as "**bons_buveurs**" which contains the tuples selected from "**bons_buveurs**"

In [164]:
%%sql
DROP TABLE IF EXISTS RBB;
--- Create RBB table
CREATE TABLE RBB AS
SELECT *
FROM bons_buveurs;

Done.
Done.


[]

In [165]:
# Test
%sql SELECT * FROM RBB LIMIT 5;

Done.


NB,NOM,PRENOM,TYPE
13,Barthes,Roland,moyen
16,Balzac,Honore de,moyen
18,Celine,Louis Ferdinand,gros
20,Chateaubriand,Francois-Rene de,moyen
27,Dumas,Alexandre,gros


#### Exercise 2.2

Update the table you used to create "**bons_buveurs**": Change the "type" to 'gros' if the total of quantities bought is over 100.

Find the instances to update (schema may differ from the one in your table)

In [166]:
%%sql
SELECT NB_MASTER2.NB, NOM, PRENOM, TYPE, SUM(QTE)
FROM NB_MASTER2
INNER JOIN NV_NB_DATES_MASTER2 ON NV_NB_DATES_MASTER2.NB=NB_MASTER2.NB
GROUP BY NB_MASTER2.NB HAVING SUM(QTE)>100 AND TYPE!='gros';

Done.


NB,NOM,PRENOM,TYPE,SUM(QTE)
2,Artaud,Antonin,moyen,583
5,Audiberti,Jacques,petit,113
9,Ajar,Emile,petit,140
44,Gide,Andre,petit,171


Update instances

In [167]:
%%sql
UPDATE NB_MASTER2
SET TYPE = 'gros'
WHERE NB IN (SELECT NB_MASTER2.NB
            FROM NB_MASTER2
            INNER JOIN NV_NB_DATES_MASTER2 ON NV_NB_DATES_MASTER2.NB=NB_MASTER2.NB
            GROUP BY NB_MASTER2.NB HAVING SUM(QTE)>100 AND TYPE!='gros')

4 rows affected.


[]

#### Exercise 2.3

Compare the content of _table_ "**RBB**" and the _view_ "**bons_buveurs**" after the update. What differences do you see? Explain

> **Answer:** The view have been modified but not the table. As we can see below the TYPE of rows with NB 2,5,9 and 44 have been modified in the view bons_buveurs.

In [168]:
%%sql
SELECT * 
FROM bons_buveurs
LIMIT 5;

Done.


NB,NOM,PRENOM,TYPE
13,Barthes,Roland,moyen
16,Balzac,Honore de,moyen
18,Celine,Louis Ferdinand,gros
20,Chateaubriand,Francois-Rene de,moyen
27,Dumas,Alexandre,gros


In [169]:
%%sql
SELECT * 
FROM RBB
LIMIT 5;

Done.


NB,NOM,PRENOM,TYPE
13,Barthes,Roland,moyen
16,Balzac,Honore de,moyen
18,Celine,Louis Ferdinand,gros
20,Chateaubriand,Francois-Rene de,moyen
27,Dumas,Alexandre,gros


#### Exercise 2.4

Create a table "**RBA**" with the same schema as "**buveurs_asec**" which contains the tuples selected from "**buveurs_asec**"

In [170]:
%%sql
DROP TABLE IF EXISTS RBA;
--- Create RBA table
CREATE TABLE RBA AS
SELECT *
FROM buveurs_asec;

Done.
Done.


[]

In [171]:
# Test
%sql SELECT * FROM RBA LIMIT 5;

Done.


NB,NOM,PRENOM,TYPE
11,Breton,Andre,petit
13,Barthes,Roland,moyen
16,Balzac,Honore de,moyen
18,Celine,Louis Ferdinand,gros
20,Chateaubriand,Francois-Rene de,moyen


#### Exercise 2.5

Insert a tuple (101, 'your last name', 'your first name', 'your type of purchases(petit, moyen, or gros)') to "**RBA**".

In [172]:
%%sql
INSERT INTO RBA (NB, NOM, PRENOM, TYPE)
VALUES (101, 'Lim', 'Pascal', 'petit');

1 rows affected.


[]

In [173]:
# Test
%sql SELECT * FROM RBA

Done.


NB,NOM,PRENOM,TYPE
11,Breton,Andre,petit
13,Barthes,Roland,moyen
16,Balzac,Honore de,moyen
18,Celine,Louis Ferdinand,gros
20,Chateaubriand,Francois-Rene de,moyen
21,Corbiere,Tristan,petit
23,Corneille,Pierre,petit
25,Char,Rene,petit
27,Dumas,Alexandre,gros
29,Fournier,Alain,petit


#### Exercise 2.6

Compare the content of _table_ "**RBA**" and the _view_ "**buveurs_asec**". What differences do you see? Explain

> **Answer:** Since we modified RBA and not the tables from which it was created, the view still shows the unmodified table. So in RBA we have one more record.

___
# PART III: Design the database schema for posts in a Social Network

In this section your task is to design the database schema for a social network app of a new startup:

The new social network will contain users, where each user will have a name, a nickname, an email, date of birth, and an address (Street, City, State, Country, Postal Code). Users can be friends of other users, and can publish posts. Each post can contain a text, date and attachment. Posts can be either original posts or replies so the app needs to handle both scenarios. When users log in, the app needs to display the posts of their friends.

**Note:** You can create diagrams of your proposal and insert them as images into this notebook.

#### Exercise 3.1

Write and explain the design of the relations of your database

> **Answer:**  
I propose to make 4 relations: 
* Users(id, name, nickname, email, date of birth, street, city, state, country, postal code) 
* Friendship(smaller_user_id, bigger_id) 
* Posts(post_id, text, date, attachment, post_type) 
* Publish(publisher_id, post_id) 

>Where I consider each user gives only one email address and only one address. Id is an primary key attribut that is assigned to a user when he enrolls. The relation Friendship represents a many-to-many relationship set where we insert smaller user id in the first column and the bigger one in the second one to avoid the redundancies.

#### Exercise 3.2

Write a view to retrieve the posts to display when a user logs in. Consider that some users may have a lot of friends and you need to limit the number of post to display. How would you select relevant posts to display first? What kind of information would you use/add in the database for this purpose? Explain your answer.

__Note:__ Limiting the number of posts just by count is too simplistic, the user could be missing something interesting to him/her.

> **Answer:** <br\>
CREATE VIEW display_view AS <br\>
SELECT Users.id AS id, Posts.post_id <br\>
FROM Users, Posts, Publish <br\>
WHERE Publish.post_id = Posts.post_id AND Publish.publisher_id <br\>
IN (SELECT Friendship.smaller_user_id FROM Friendship WHERE Friendship.bigger_id = Users.id) <br\>
OR IN (SELECT Friendship.bigger_user_id FROM Friendship WHERE Friendship.smaller_id = Users.id) 

>To select relevant posts to display first, I propose to (1) add the number that one post has been clicked or reposted by other users as a new attribute in the relation Posts. (2) add the number of reactions (like replying and like) between two users in the relation Friendship. I think this kind of information can help us know the importance of a post to a specific user, we can then display to him the most relevant posts first.